# ReAct ToolSelection approach with Azure AI Search
このサンプルノートブックでは ReAct を用いてツール選択の動作を試すことができます。ユーザーからの質問に対してどのような情報が欠けているのかを確認するために質問を繰り返し評価し、すべての情報がそろったところで、回答を作成することを試みます。ReAct を使用してツールの「**説明文**」のみに基づいて使用するツールを決定します。

サンプルコードではツールを2つ（Azure AI Search、CSVルックアップ）使用して情報を検索しています。エージェントの処理は LangChain の**ZERO_SHOT_REACT_DESCRIPTION** エージントで実装しています。サンプルコードは武将カフェの検索用 CSV ファイルから検索するようになっています。


# 事前準備
この Python サンプルを実行するには、以下が必要です：
- Azure AI Search リソース。エンドポイントとクエリ API キーが必要です。
- Azure OpenAI Service にアクセスできる承認済み Azure サブスクリプション
- Azure OpenAI Service への `text-embedding-ada-002` Embeddings モデルのデプロイメント。このデモでは、API バージョン `2023-05-15` を使用しています。デプロイ名はモデルと同じ「`text-embedding-ada-002`」を使用しています。
- Azure OpenAI Service の接続とモデル情報
  - OpenAI API キー
  - OpenAI Embeddings モデルのデプロイメント名
  - OpenAI API バージョン
- Python (この手順はバージョン 3.10.x でテストされています)

これらのデモには、Visual Studio Code と [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) を使用できます。

## パッケージのインストール

In [ ]:
!pip install azure-search-documents==11.4.0
!pip install openai
!pip install langchain==0.0.350

In [1]:
import azure.search.documents
print("azure.search.documents", azure.search.documents.__version__)
import langchain
print("langchain", langchain.__version__)

azure.search.documents 11.4.0
langchain 0.0.352


## 必要なライブラリと環境変数のインポート

In [2]:
import csv
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    VectorizedQuery
)

from langchain.agents import (
    AgentType,
    Tool,
    initialize_agent,
)
from langchain.agents.mrkl import prompt
from langchain.chat_models import AzureChatOpenAI
#from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool

## Azure AI Search 接続設定

In [3]:
service_endpoint: str = "<Your search service endpoint>"
service_query_key: str = "<Your search service query key>"
index_name: str = "gptkbindex" #自動構築時のデフォルト設定

credential = AzureKeyCredential(service_query_key)

## Azure OpenAI の設定

In [4]:
AZURE_OPENAI_API_KEY = "Your OpenAI API Key"
AZURE_OPENAI_ENDPOINT = "https://<Your OpenAI Service>.openai.azure.com/"
AZURE_OPENAI_CHATGPT_DEPLOYMENT = "chat" #自動構築時のデフォルト設定
AZURE_OPENAI_EMB_DEPLOYMENT="embedding" #自動構築時のデフォルト設定

In [5]:
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt  

openai_client = AzureOpenAI(
  api_key = AZURE_OPENAI_API_KEY,  
  api_version = "2023-05-15",
  azure_endpoint = AZURE_OPENAI_ENDPOINT
)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# タイトルフィールドとコンテンツフィールドのEmbeddingsを生成する関数。
def generate_embeddings(text, model=AZURE_OPENAI_EMB_DEPLOYMENT):
    return openai_client.embeddings.create(input = [text], model=model).data[0].embedding

# 武将検索ツールの Retrieve 実装
武将検索ツールは Azure AI Search に接続して検索した結果を返却します。

In [16]:
def retrieve(query_text: str):
    search_client = SearchClient(service_endpoint, index_name, credential=credential)
    docs = search_client.search(
        search_text=query_text,
        filter=None,
        top=3,
        vector_queries=[VectorizedQuery(vector=generate_embeddings(query_text), k_nearest_neighbors=3, fields="embedding")]
    )
    results =[doc['sourcepage'] + ": " + nonewlines(doc['content']) for doc in docs]
    content = "\n".join(results)
    return content
    
def nonewlines(s: str) -> str:
    return s.replace('\n', ' ').replace('\r', ' ').replace('[', '【').replace(']', '】')

# カフェ検索ツールの定義
カフェ検索ツールは、CSV ルックアップを使用してクエリに該当するカフェの情報を返却します。

In [17]:
class CafeSearchTool(BaseTool):
    data: dict[str, str] = {}
    name = "CafeSearchTool"
    description = "武将のゆかりのカフェを検索するのに便利です。カフェの検索クエリには、武将の**名前のみ**を入力してください。"

    # Use the tool synchronously.
    def _run(self, query: str) -> str:
        filename = "data/restaurantinfo.csv"
        key_field = "name"
        try:
            with open(filename, newline='', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    self.data[row[key_field]] =  "\n".join([f"{i}:{row[i]}" for i in row])

        except Exception as e:
            print("File read error:", e)

        return self.data.get(query, "")

# Tool の定義
使用するツールは以下の2つで、それぞれ異なる2種類の記法を使って記述しています。
- 武将検索ツール: Tool dataclass 法を使って tools の中に直接記述。
- カフェ検索ツール: BaseTool クラスのサブクラスとして `CafeSearchTool` を定義

In [18]:
# Tool dataclass 法と Subclassing the BaseTool class 法の異なる記法を示しています
tools = [
    Tool(name="PeopleSearchTool",
        func=retrieve,
        coroutine=retrieve,
        description="日本の歴史の人物情報の検索に便利です。ユーザーの質問から検索クエリーを生成して検索します。クエリーは文字列のみを受け付けます"
        ),
    CafeSearchTool()
]

# LLM の定義

In [19]:
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    api_version="2023-07-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_key=AZURE_OPENAI_API_KEY,
    #azure_ad_token_provider=self.openai_ad_token,
    temperature=0.0,
)

In [20]:
q = "鎌倉幕府第二代征夷大将軍の名前とその将軍にゆかりの地にあるカフェの名前を教えて"

# Agent の定義と実行

In [22]:
SUFFIX = """
Answer should be in Japanese.
"""
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    #agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=dict(suffix=SUFFIX + prompt.SUFFIX),
    handle_parsing_errors=True,
    max_iterations=5,
    early_stopping_method="generate",
)

result = agent_chain.run(q)
result



> Entering new AgentExecutor chain...
まずは鎌倉幕府第二代征夷大将軍の名前を調べる必要があります。それから、その将軍にゆかりの地にあるカフェの名前を検索します。
Action: PeopleSearchTool
Action Input: 鎌倉幕府第二代征夷大将軍
Observation: 源頼家 - Wikipedia-1.pdf: 第二代鎌倉殿 建久10年（1199年）1月13日、父・頼朝が急死する。頼家は同月20日付けで左中将となり、つい で26日付けで家督を相続し、第2代鎌倉殿となる。時に18歳であった。1 - 2月頃には武士達が大勢 京都に上り、急な政権交代に乗じた都の不穏な動きを警戒する態勢が取られており、この間に三 左衛門事件が発生している。 頼家が家督を相続して3か月後の4月、北条氏ら有力御家人による十三人の合議制がしかれ、頼家 が直に訴訟を聴断することは停止された。反発した頼家は小笠原長経、比企三郎、比企時員、中 野能成以下若い近習5人【注釈2】を指名して、彼らでなければ自分への目通りを許さず - 、 またこれに 手向かってはならないという命令を出した。また正治元年（1199年）7月には小笠原、比企、中 野、和田朝盛らに対して、安達景盛の留守を狙い、その愛妾を召し連れて来るように命じた。こ のあたりの『吾妻鏡』には、頼家が側近や乳母一族である比企氏を重用し、従来の慣習を無視し た独裁的判断を行った挿話が並べられている（十三人の合議制の実態や頼家が本当に暗君であっ たかについては、#十三人の合議制の実態と頼家の実績を参照）。 - 合議制の設立から半年後の10月、頼朝の代から側近として重用されていた侍所長官の梶原景時に 反発する御家人たちには、御家人66名による景時糾弾の連判状を頼家に提出した。頼家に弁明を 求められた景時は、何の抗弁もせず所領に下る。謹慎ののち、鎌倉へ戻った景時は政務への復帰 - を頼家に願ったが、頼家は景時を救うことができず 、 景時は鎌倉追放を申し渡された。正治2年 （1200年）1月20日、失意の景時は一族を率いて京都へ上る道中で在地の御家人達から襲撃を受 け、一族もろとも滅亡した（『吾妻鏡』）。九条兼実の『玉葉』正治2年正月2日条によると、景 時は頼家の弟である千幡（のちの源実朝）を将

'鎌倉幕府第二代征夷大将軍の名前は源頼家であり、その将軍にゆかりの地にあるカフェの名前は「Cafe Genji13」です。'